In [1]:
#| default_exp ior

In [2]:
#| export
import numpy as np
from PIL import Image

In [3]:
#| export
def insalient_outsalient_ratio_defunc(reference, comparative, object_mask):
    assert(reference.shape == comparative.shape)
#     ref_tmp = Image.fromarray(reference.squeeze()).resize(comparative.shape)
#     ref = np.array(ref_tmp)

    agreement = reference * comparative
#     plt.imshow(inter_model_saliency)

    insalient_agreement = (agreement * object_mask)
    insalient_agreement_norm = np.exp(insalient_agreement) / np.exp(insalient_agreement.max())
    insalient_agreement_norm = np.where(insalient_agreement_norm >= 0.9, 1, insalient_agreement_norm)

    reference_in_distribution = reference * object_mask
    reference_in_distribution = reference_in_distribution / reference_in_distribution.sum()

    insalient = np.sum(insalient_agreement_norm * reference_in_distribution)
    insalient = np.log(insalient)

    outsalient_agreement = (agreement * np.logical_not(object_mask).astype(float))
    outsalient_agreement_norm = np.exp(outsalient_agreement) / np.exp(outsalient_agreement.max())
    outsalient_agreement_norm = np.where(outsalient_agreement_norm >= 0.9, 1, outsalient_agreement_norm)

    reference_out_distribution = reference * np.logical_not(object_mask).astype(float)
    reference_out_distribution = reference_out_distribution / reference_out_distribution.sum()

    outsalient = np.sum(outsalient_agreement_norm * reference_out_distribution)
    outsalient = np.log(outsalient)

    return insalient / (insalient + outsalient)

In [6]:
#| export
def insalient_outsalient_ratio_v2(reference, comparative, object_mask):
    """
    Calculated IOR metric based on the reference and comparative saliency maps,
    and in lieu of the mask of the object
    
    input :
      - reference : ndarray (Reference saliency map as the focus of the metric)
      - comparative : ndarray (Comparative to compare the reference saliency map to)
      - object_mask : ndarray (Binary mask with 1 in the region within the object and 0 elsewhere)
      
    return :
      - ior : ndarray (Final score of the IOR metric)
    """
    assert(reference.shape == comparative.shape)
    agreement = reference * comparative

    epsilon = 1e-8
    insalient_agreement = (agreement * object_mask) + epsilon
    
    reference_distribution = reference + epsilon
    reference_distribution = reference_distribution / reference_distribution.sum()
    
    reference_in_distribution = (reference_distribution * object_mask)

    # Expect in-saliency agreement to be in the same place. Also the value that is concentrated in the reference must be the one with greatest agreement as well.
    insalient = np.sum(insalient_agreement * reference_in_distribution)

    # expect the saliency outside mask to be spurious and spread out and therefore, not coincide with each other in both saliencies
    outsalient_agreement = (reference * np.logical_not(object_mask).astype(float)) + epsilon
    
    reference_out_distribution = reference_distribution * np.logical_not(object_mask).astype(float)

    # out-salient distribution must also not have large values
    outsalient = np.sum(outsalient_agreement * reference_out_distribution)

    return insalient/ (outsalient + insalient)

In [1]:
#| hide
from nbdev.export import nb_export
nb_export('ior_metric.ipynb', '../../commons/api/metric')